In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('contains_null').getOrCreate()

23/12/13 20:23:36 WARN Utils: Your hostname, DESKTOP-0HJMRJF resolves to a loopback address: 127.0.1.1; using 172.19.51.88 instead (on interface eth0)
23/12/13 20:23:36 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/13 20:23:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/12/13 20:23:38 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
spark

In [4]:
df = spark.read.csv('ContainsNull.csv', inferSchema=True, header=True)
df.show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| NULL|
|emp2| NULL| NULL|
|emp3| NULL|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+


In [18]:
df.na.drop().show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp4|Cindy|456.0|
+----+-----+-----+


In [19]:
df.na.drop(thresh=1).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| NULL|
|emp2| NULL| NULL|
|emp3| NULL|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+


In [20]:
df.na.drop(how='all').show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| NULL|
|emp2| NULL| NULL|
|emp3| NULL|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+


In [22]:
df.na.drop(subset='Sales').show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp3| NULL|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+


In [44]:
df.na.fill('Test').show()
df.na.fill(997).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| NULL|
|emp2| Test| NULL|
|emp3| Test|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|997.0|
|emp2| NULL|997.0|
|emp3| NULL|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+


In [46]:
df.na.fill('997', 'Name').show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| NULL|
|emp2|  997| NULL|
|emp3|  997|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+


In [47]:
df.na.fill('NaN', subset=['Name']).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| NULL|
|emp2|  NaN| NULL|
|emp3|  NaN|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+


In [71]:
from pyspark.sql.functions import min, mean

In [72]:
mean_val = df.select(mean(df['Sales']).alias('Mean')).collect()

In [73]:
mean_val = mean_val[0]['Mean']

In [74]:
df.na.fill(mean_val, subset=['Sales']).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|400.5|
|emp2| NULL|400.5|
|emp3| NULL|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+


In [75]:
df.na.fill(df.select(mean(df['Sales'])).collect()[0][0], subset=['Sales']).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|400.5|
|emp2| NULL|400.5|
|emp3| NULL|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+
